In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('../data/beta_event_CGID.csv')

,Unnamed: 0,Trial,Hit/Miss,Peak Frequency,Lower Frequency Bound,Upper Frequency Bound,Frequency Span,Peak Time,Event Onset Time,Event Offset Time,Event Duration,Peak Power,Normalized Peak Power,Outlier Event,ob,obpres,contact,grip,event_name,window
0,2,0,23.884833,16,14,19,5,0.127,0.024,0.223,0.199,2143.564772,18.817809,True,1,20.822300,24.728367,2,go,"(-0.5, 0)"
1,3,0,23.884833,19,16,24,8,0.289,0.001,0.378,0.377,1197.378987,8.855848,True,1,20.822300,24.728367,2,go,"(-0.5, 0)"
2,4,0,23.884833,20,17,24,7,0.309,0.226,0.387,0.161,1184.063002,9.096938,True,1,20.822300,24.728367,2,go,"(-0.5, 0)"
3,62,4,100.665633,19,15,23,8,0.326,0.260,0.442,0.182,1204.699499,8.909990,True,1,97.603100,101.571700,2,go,"(-0.5, 0)"
4,94,6,146.134067,21,17,25,8,0.090,0.034,0.157,0.123,741.820111,6.013198,True,0,143.102900,146.774067,3,go,"(-0.5, 0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,1436,104,1264.829900,17,14,22,8,0.174,0.041,0.252,0.211,832.761397,9.369597,True,0,1264.829900,1265.814600,3,obpres,"(0, 0.5)"
267,1469,106,1411.594600,22,18,26,8,0.091,0.024,0.159,0.135,554.479871,7.276662,True,1,1411.594600,1412.985567,1,obpres,"(0, 0.5)"
268,1470,106,1411.594600,23,19,27,8,0.076,0.001,0.162,0.161,545.902963,7.851516,True,1,1411.594600,1412.985567,1,obpres,"(0, 0.5)"
269,1544,111,1484.156667,26,22,33,11,0.186,0.126,0.228,0.102,374.137975,6.088310,True,0,1484.156667,1485.610100,1,obpres,"(0, 0.5)"
